In [ ]:
!wget https://raw.githubusercontent.com/marcin119a/benchmark_tests/main/data/long_dna_5.txt

--2023-07-24 18:32:58--  https://raw.githubusercontent.com/marcin119a/benchmark_tests/main/data/long_dna_5.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414708 (405K) [text/plain]
Saving to: ‘long_dna_5.txt’

long_dna_5.txt      100%[===================>] 404.99K  --.-KB/s    in 0.03s   

2023-07-24 18:32:59 (11.8 MB/s) - ‘long_dna_5.txt’ saved [414708/414708]



In [ ]:
def count_lines_in_file(file_name):
    # Inicjalizacja licznika linii
    line_count = 0

    # Otwarcie pliku w trybie do odczytu
    with open(file_name, 'r') as file:
        for line in file:
            line_count += 1

    return line_count

file_name = 'long_dna_5.txt'
lines_count = count_lines_in_file(file_name)
print(f"Liczba linii w pliku '{file_name}': {lines_count}")


Liczba linii w pliku 'long_dna_5.txt': 29622


In [ ]:
def parse_data_file(data_path, num_samples):
    # Inicjalizacja list dla danych wejściowych i wyjściowych oraz zbiorów znaków
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()

    # Wczytanie danych z pliku txt
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')

    # Przetwarzanie danych i zbiorów znaków
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text = line.split(' ')
        # Dodanie znaków startu i końca dla target_text
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        # Aktualizacja zbiorów znaków
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

    return input_texts, target_texts, input_characters, target_characters

num_samples = 20000
data_path = 'long_dna_5.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 20000
Liczba danych wyjściowych: 20000
Zbiór znaków wejściowych: {'N', 'T', 'C', 'A', 'G'}
Zbiór znaków wyjściowych: {'N', 'T', 'C', '\t', 'A', '\n', 'G'}


In [ ]:
input_characters

{'A', 'C', 'G', 'N', 'T'}

In [ ]:
target_characters

{'\t', '\n', 'A', 'C', 'G', 'N', 'T'}

In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np



latent_dim = 256  # Latent dimensionality of the encoding space.

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
import numpy as np

def create_token_index(tokens):
    token_index = dict([(char, i) for i, char in enumerate(tokens)])
    return token_index

def transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length):
    num_samples = len(input_texts)
    num_encoder_tokens = len(input_token_index)
    num_decoder_tokens = len(target_token_index)

    encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.

    return encoder_input_data, decoder_input_data, decoder_target_data

# Przykład użycia funkcji
input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of samples: 20000
Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 6
Max sequence length for outputs: 8
Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 6
Max sequence length for outputs: 8


In [ ]:
np.savez_compressed('decoder_target_data.npz', decoder_target_data=decoder_target_data)

In [ ]:
np.savez_compressed('decoder_input_data.npz', decoder_input_data=decoder_input_data)

In [ ]:
np.savez_compressed('encoder_input_data.npz', encoder_input_data=encoder_input_data)

In [ ]:
batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Epoch 1/100
125/125 [==============================] - 11s 14ms/step - loss: 0.7178 - accuracy: 0.5785 - val_loss: 0.3274 - val_accuracy: 0.7545
Epoch 2/100
125/125 [==============================] - 1s 7ms/step - loss: 0.2259 - accuracy: 0.7848 - val_loss: 0.1885 - val_accuracy: 0.7895
Epoch 3/100
125/125 [==============================] - 1s 6ms/step - loss: 0.1940 - accuracy: 0.7879 - val_loss: 0.1821 - val_accuracy: 0.7877
Epoch 4/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1797 - accuracy: 0.7900 - val_loss: 0.1790 - val_accuracy: 0.7908
Epoch 5/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1786 - accuracy: 0.7904 - val_loss: 0.1772 - val_accuracy: 0.7898
Epoch 6/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1747 - accuracy: 0.7907 - val_loss: 0.1715 - val_accuracy: 0.7911
Epoch 7/100
125/125 [==============================] - 1s 7ms/step - loss: 0.1757 - accuracy: 0.7913 - val_loss: 0.1753 - val_accuracy: 0.79

In [ ]:

def matching_position_k_mers(result1, result2, space = True):

  # Initialise counter of compatible items
  matching_positions = 0
  if space:
    result2 = ' ' + result2
  # Compare the letters in each position
  for i in range(len(result1)):
      if result1[i] == result2[i]:
          matching_positions += 1

  return {matching_positions}

matching_position_k_mers('GTTTTGCAGCCG', 'TTTTGCAGCCGA')


{11}

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(input_texts[seq_index], decoded_sentence))

1/1 [==============================] - 0s 376ms/step
-
Input sentence: TATAAT
Decoded sentence: ATAATG

On how many positions they agree on {5}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ATAATT
Decoded sentence: TAATTT

On how many positions they agree on {5}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TAATTA
Decoded sentence: AATTAC

On how many positions they agree on {5}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: AATTAA
Decoded sentence: ATTAAT

On how many positions they agree on {5}
1/1 [==============================] - 0s 22ms/step
-
Input sentence: ATTAAT
Decoded sentence: TTAATG

On how many positions they agree on {5}
1/1 [==============================] - 0s 18ms/step
-
Input sentence: TTAATA
Decoded sentence: TAATAC

On how many positions they agree on {5}
1/1 [==============================] - 0s 19ms/step
-
Input sentence: TAATAA
Decoded sentence: AATAAC

On how many positions they agree on {5

In [ ]:
!wget https://raw.githubusercontent.com/marcin119a/benchmark_tests/main/data/long_dna_errors_5.txt

--2023-07-24 18:37:15--  https://raw.githubusercontent.com/marcin119a/benchmark_tests/main/data/long_dna_errors_5.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414358 (405K) [text/plain]
Saving to: ‘long_dna_errors_5.txt’

long_dna_errors_5.t 100%[===================>] 404.65K  --.-KB/s    in 0.03s   

2023-07-24 18:37:15 (11.4 MB/s) - ‘long_dna_errors_5.txt’ saved [414358/414358]



### Corrected ERRORS by trained encoder-decoder network

In [ ]:
file_name = 'long_dna_errors_5.txt'
lines_count = count_lines_in_file(file_name)
print(f"Liczba linii w pliku '{file_name}': {lines_count}")

Liczba linii w pliku 'long_dna_errors_5.txt': 29597


In [ ]:
num_samples = 20000
data_path = 'long_dna_errors_5.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 20000
Liczba danych wyjściowych: 20000
Zbiór znaków wejściowych: {'N', 'T', 'C', 'A', 'G'}
Zbiór znaków wyjściowych: {'N', 'T', 'C', '\t', 'A', '\n', 'G'}


In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)




Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 6
Max sequence length for outputs: 8


In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=300,
          validation_split=0.2)

Epoch 1/300
125/125 [==============================] - 1s 8ms/step - loss: 0.9968 - accuracy: 0.5552 - val_loss: 0.6506 - val_accuracy: 0.6415
Epoch 2/300
125/125 [==============================] - 1s 7ms/step - loss: 0.5371 - accuracy: 0.6949 - val_loss: 0.4770 - val_accuracy: 0.7081
Epoch 3/300
125/125 [==============================] - 1s 7ms/step - loss: 0.4481 - accuracy: 0.7202 - val_loss: 0.4271 - val_accuracy: 0.7287
Epoch 4/300
125/125 [==============================] - 1s 7ms/step - loss: 0.4026 - accuracy: 0.7320 - val_loss: 0.3806 - val_accuracy: 0.7302
Epoch 5/300
125/125 [==============================] - 1s 7ms/step - loss: 0.3597 - accuracy: 0.7345 - val_loss: 0.3550 - val_accuracy: 0.7311
Epoch 6/300
125/125 [==============================] - 1s 7ms/step - loss: 0.3371 - accuracy: 0.7369 - val_loss: 0.3427 - val_accuracy: 0.7313
Epoch 7/300
125/125 [==============================] - 1s 7ms/step - loss: 0.3225 - accuracy: 0.7390 - val_loss: 0.3281 - val_accuracy: 0.7368

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s_erros.h5')

In [ ]:

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())



for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Ground truth:', target_texts[seq_index])
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(target_texts[seq_index], decoded_sentence))

1/1 [==============================] - 1s 827ms/step


-
Ground truth: 	GTTCTN

Input sentence: CGTTCT
Decoded sentence: CGTTCT

On how many positions they agree on {2}
1/1 [==============================] - 0s 28ms/step
-
Ground truth: 	TTCTNA

Input sentence: GTTCTN
Decoded sentence: GTTCTT

On how many positions they agree on {2}
1/1 [==============================] - 0s 19ms/step
-
Ground truth: 	TCTNAA

Input sentence: TTCTNA
Decoded sentence: TTCTTA

On how many positions they agree on {3}
1/1 [==============================] - 0s 20ms/step
-
Ground truth: 	CTNAAA

Input sentence: TCTNAA
Decoded sentence: TCTTAA

On how many positions they agree on {3}
1/1 [==============================] - 0s 19ms/step
-
Ground truth: 	TNAAAA

Input sentence: CTNAAA
Decoded sentence: CTTAAA

On how many positions they agree on {4}
1/1 [==============================] - 0s 20ms/step
-
Ground truth: 	NAAAAG

Input sentence: TNAAAA
Decoded sentence: TTAAAA

On how many positions they agree on {4}
1/1 [==============================] - 0s 20ms/step
-
Gr

In [ ]:
def filter_lines_with_character(input_file, output_file, character='N'):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            # Podziel linię na kolumny (oddzielone spacją)
            column1, column2 = line.strip().split(' ')
            # Sprawdź, czy w którejś kolumnie występuje znak 'N'
            if character in column1 or character in column2:
                # Jeśli tak, zapisz linię w pliku wynikowym
                f_out.write(line)

# Przykład użycia funkcji
input_file = 'long_dna_5.txt'
output_file = 'errors.txt'
filter_lines_with_character(input_file, output_file, character='N')


In [ ]:
num_samples = 10000
data_path = 'errors.txt'
input_texts, target_texts, input_characters, target_characters = parse_data_file(data_path, num_samples)

# Wyświetlenie informacji o danych
print("Liczba danych wejściowych:", len(input_texts))
print("Liczba danych wyjściowych:", len(target_texts))
print("Zbiór znaków wejściowych:", input_characters)
print("Zbiór znaków wyjściowych:", target_characters)


Liczba danych wejściowych: 29
Liczba danych wyjściowych: 29
Zbiór znaków wejściowych: {'N', 'T', 'C', 'A', 'G'}
Zbiór znaków wyjściowych: {'N', 'T', 'C', '\t', 'A', '\n', 'G'}


In [ ]:
from __future__ import print_function
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

input_token_index = create_token_index(input_characters)
target_token_index = create_token_index(target_characters)
encoder_input_data, decoder_input_data, decoder_target_data = transform_data(input_texts, target_texts, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)

# Wyświetlenie informacji o danych
print("Number of unique input tokens:", len(input_token_index))
print("Number of unique output tokens:", len(target_token_index))
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)




Number of unique input tokens: 5
Number of unique output tokens: 7
Max sequence length for inputs: 6
Max sequence length for outputs: 8


In [ ]:
input_token_index.shape

AttributeError: ignored

In [ ]:
for seq_index in range(29):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('On how many positions they agree on', matching_position_k_mers(input_texts[seq_index], decoded_sentence, space = False))

1/1 [==============================] - 0s 19ms/step
-
Input sentence: CGTTCT
Decoded sentence: CGTTCT

On how many positions they agree on {6}
1/1 [==============================] - 0s 19ms/step
-
Input sentence: GTTCTN
Decoded sentence: GTTCTT

On how many positions they agree on {5}
1/1 [==============================] - 0s 22ms/step
-
Input sentence: TTCTNA
Decoded sentence: TTCTTA

On how many positions they agree on {5}
1/1 [==============================] - 0s 22ms/step
-
Input sentence: TCTNAA
Decoded sentence: TCTTAA

On how many positions they agree on {5}
1/1 [==============================] - 0s 25ms/step
-
Input sentence: CTNAAA
Decoded sentence: CTTAAA

On how many positions they agree on {5}
1/1 [==============================] - 0s 57ms/step
-
Input sentence: TNAAAA
Decoded sentence: TTAAAA

On how many positions they agree on {5}
1/1 [==============================] - 0s 74ms/step
-
Input sentence: NAAAAG
Decoded sentence: GAAAAG

On how many positions they agree on {5}